# Ai4i2020 - Predictive maintenance

####  Problem Statement : To predict the Air temperature when there is machine failure in industry due to following failure mode
    - tool wear failure (TWF)
    - heat dissipation failure (HDF)
    - power failure (PWF)
    - verstrain failure (OSF)
    - random failures (RNF)


## Import reqired libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge,Lasso,LassoCV, RidgeCV, LarsCV,ElasticNet,ElasticNetCV
import statsmodels.api as sm
from datetime import datetime
import logging
import pickle
import os
os.chdir(os.getcwd())
logging.basicConfig(filename='Ai4i_2020.log',level = logging.INFO,force = True)
logging.info("Ai4i 2020 Predictive Maintenance")

## EDA

In [2]:
df = pd.read_csv("ai4i2020.csv")

In [3]:
df

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,0,0,0,0,0


In [4]:
pf = pp.ProfileReport(df)
pd_f = pf.to_widgets()
pf.to_file("AI4I_Profiling.html")

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Data Preprocessing

In [4]:
df = df.drop(['UDI','Product ID','Type','Machine failure'], axis = 1)
df.columns = df.columns.str.replace('[',"",regex=False)
df.columns = df.columns.str.replace(']',"",regex=False)
df.columns = df.columns.str.replace(' ',"",regex=False)

In [5]:
df

,AirtemperatureK,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,TWF,HDF,PWF,OSF,RNF
0,298.1,308.6,1551,42.8,0,0,0,0,0,0
1,298.2,308.7,1408,46.3,3,0,0,0,0,0
2,298.1,308.5,1498,49.4,5,0,0,0,0,0
3,298.2,308.6,1433,39.5,7,0,0,0,0,0
4,298.2,308.7,1408,40.0,9,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,0,0,0,0,0
9996,298.9,308.4,1632,31.8,17,0,0,0,0,0
9997,299.0,308.6,1645,33.4,22,0,0,0,0,0
9998,299.0,308.7,1408,48.5,25,0,0,0,0,0


In [6]:
df.to_csv("Cleaned_ai4i.csv")

## Separate independent and target variable

In [7]:
X = df.drop("AirtemperatureK", axis=1)

In [8]:
X

,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,TWF,HDF,PWF,OSF,RNF
0,308.6,1551,42.8,0,0,0,0,0,0
1,308.7,1408,46.3,3,0,0,0,0,0
2,308.5,1498,49.4,5,0,0,0,0,0
3,308.6,1433,39.5,7,0,0,0,0,0
4,308.7,1408,40.0,9,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
9995,308.4,1604,29.5,14,0,0,0,0,0
9996,308.4,1632,31.8,17,0,0,0,0,0
9997,308.6,1645,33.4,22,0,0,0,0,0
9998,308.7,1408,48.5,25,0,0,0,0,0


In [9]:
y = df['AirtemperatureK']

In [10]:
y

0       298.1
1       298.2
2       298.1
3       298.2
4       298.2
        ...  
9995    298.8
9996    298.9
9997    299.0
9998    299.0
9999    299.0
Name: AirtemperatureK, Length: 10000, dtype: float64

## Data PreProcessing - Standardization

In [11]:
scaler = StandardScaler()
arr = scaler.fit_transform(X)

In [12]:
arr

array([[-0.94735989,  0.06818514,  0.28219976, ..., -0.09793424,
        -0.09948362, -0.04363046],
       [-0.879959  , -0.72947151,  0.63330802, ..., -0.09793424,
        -0.09948362, -0.04363046],
       [-1.01476077, -0.22744984,  0.94428963, ..., -0.09793424,
        -0.09948362, -0.04363046],
       ...,
       [-0.94735989,  0.59251888, -0.66077672, ..., -0.09793424,
        -0.09948362, -0.04363046],
       [-0.879959  , -0.72947151,  0.85400464, ..., -0.09793424,
        -0.09948362, -0.04363046],
       [-0.879959  , -0.2162938 ,  0.02137647, ..., -0.09793424,
        -0.09948362, -0.04363046]])

In [13]:
df1 = pd.DataFrame(arr)

In [14]:
df1.columns = X.columns
df1

,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,TWF,HDF,PWF,OSF,RNF
0,-0.947360,0.068185,0.282200,-1.695984,-0.06798,-0.10786,-0.097934,-0.099484,-0.04363
1,-0.879959,-0.729472,0.633308,-1.648852,-0.06798,-0.10786,-0.097934,-0.099484,-0.04363
2,-1.014761,-0.227450,0.944290,-1.617430,-0.06798,-0.10786,-0.097934,-0.099484,-0.04363
3,-0.947360,-0.590021,-0.048845,-1.586009,-0.06798,-0.10786,-0.097934,-0.099484,-0.04363
4,-0.879959,-0.729472,0.001313,-1.554588,-0.06798,-0.10786,-0.097934,-0.099484,-0.04363
...,...,...,...,...,...,...,...,...,...
9995,-1.082162,0.363820,-1.052012,-1.476034,-0.06798,-0.10786,-0.097934,-0.099484,-0.04363
9996,-1.082162,0.520005,-0.821283,-1.428902,-0.06798,-0.10786,-0.097934,-0.099484,-0.04363
9997,-0.947360,0.592519,-0.660777,-1.350349,-0.06798,-0.10786,-0.097934,-0.099484,-0.04363
9998,-0.879959,-0.729472,0.854005,-1.303217,-0.06798,-0.10786,-0.097934,-0.099484,-0.04363


In [15]:
df1.describe()

,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,TWF,HDF,PWF,OSF,RNF
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
mean,3.841483e-15,-2.554568e-16,5.259244e-16,4.147793e-17,1.447314e-16,3.394757e-16,2.058548e-16,4.610007e-16,1.075350e-15
std,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00
min,-2.901986e+00,-2.068196e+00,-3.630149e+00,-1.695984e+00,-6.797983e-02,-1.078600e-01,-9.793424e-02,-9.948362e-02,-4.363046e-02
25%,-8.125581e-01,-6.458012e-01,-6.808401e-01,-8.633176e-01,-6.797983e-02,-1.078600e-01,-9.793424e-02,-9.948362e-02,-4.363046e-02
50%,6.365340e-02,-1.995597e-01,1.134481e-02,7.698234e-04,-6.797983e-02,-1.078600e-01,-9.793424e-02,-9.948362e-02,-4.363046e-02
75%,7.376623e-01,4.084443e-01,6.834663e-01,8.491466e-01,-6.797983e-02,-1.078600e-01,-9.793424e-02,-9.948362e-02,-4.363046e-02
max,2.557486e+00,7.514840e+00,3.672902e+00,2.278819e+00,1.471024e+01,9.271274e+00,1.021093e+01,1.005191e+01,2.291977e+01


## Check Multicolliarity

In [11]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [13]:
vif = pd.DataFrame()
vif['vif_factor'] =  [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['vif_predictors'] = X.columns
vif

,vif_factor,vif_predictors
0,774.059340,ProcesstemperatureK
1,379.790129,Rotationalspeedrpm
2,87.940856,TorqueNm
3,4.031034,Toolwearmin
4,1.020261,TWF
5,1.032956,HDF
6,1.221416,PWF
7,1.092945,OSF
8,1.003371,RNF


- we found multicollinearity in MAchinefailure column VIF > 10, dropping feature "Machinefailure"

## Splitting the data

In [15]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=100)

In [16]:
x_train

,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,TWF,HDF,PWF,OSF,RNF
8369,309.7,1953,20.6,25,0,0,0,0,0
9722,309.8,1911,21.6,126,0,0,0,0,0
6950,311.3,1374,53.1,66,0,0,0,0,0
1919,308.0,1658,30.0,3,0,0,0,0,0
5713,311.9,1596,38.0,156,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
350,308.3,1400,44.3,43,0,0,0,0,0
79,308.9,1402,37.9,2,0,0,0,0,0
8039,311.9,1560,31.8,21,0,0,0,0,0
6936,311.1,1471,40.7,26,0,0,0,0,0


In [17]:
x_test

,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,TWF,HDF,PWF,OSF,RNF
8018,311.9,1889,23.5,198,0,0,0,0,0
9225,309.1,1820,23.6,120,0,0,0,0,0
3854,311.0,1338,67.6,194,0,0,1,1,0
2029,308.5,1668,35.2,77,0,0,0,0,0
3539,311.0,1686,32.5,25,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
6923,311.7,1433,49.3,206,0,0,0,0,0
1207,308.1,1573,34.9,93,0,0,0,0,0
7960,312.2,1491,42.6,52,0,0,0,0,0
2339,308.2,1466,41.3,8,0,0,0,0,0


In [18]:
y_train

8369    298.8
9722    298.8
6950    300.7
1919    298.2
5713    302.2
        ...  
350     297.6
79      298.8
8039    300.8
6936    300.6
5640    302.6
Name: AirtemperatureK, Length: 8000, dtype: float64

In [19]:
y_test

8018    301.0
9225    298.0
3854    302.4
2029    298.7
3539    302.0
        ...  
6923    301.2
1207    297.0
7960    301.1
2339    299.2
6637    301.5
Name: AirtemperatureK, Length: 2000, dtype: float64

## Training the model

In [20]:
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

In [21]:
lr.intercept_

-62.723732145575184

In [22]:
lr.coef_

array([ 1.16950956e+00,  1.11034469e-04, -7.00948789e-04,  1.27828249e-04,
        1.40637939e-01,  1.68510575e+00,  1.37489422e-01, -1.44659251e-01,
       -6.58611734e-02])

## Save linear model

In [23]:
pickle.dump(lr,open('Ai4i_predictive_Maintenance.pickle','wb'))

In [24]:
lm = pickle.load(open('Ai4i_predictive_Maintenance.pickle','rb'))

In [25]:
lr_score = lm.score(x_test,y_test)
lr_score

0.7965460964724067

In [26]:
df

,AirtemperatureK,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,TWF,HDF,PWF,OSF,RNF
0,298.1,308.6,1551,42.8,0,0,0,0,0,0
1,298.2,308.7,1408,46.3,3,0,0,0,0,0
2,298.1,308.5,1498,49.4,5,0,0,0,0,0
3,298.2,308.6,1433,39.5,7,0,0,0,0,0
4,298.2,308.7,1408,40.0,9,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,0,0,0,0,0
9996,298.9,308.4,1632,31.8,17,0,0,0,0,0
9997,299.0,308.6,1645,33.4,22,0,0,0,0,0
9998,299.0,308.7,1408,48.5,25,0,0,0,0,0


In [28]:
lr.predict([[308.6,1551,42.8,0,0,0,0,0,0]])

array([298.3291309])

In [29]:
lm.predict([[308.7,1500,40.2,30,0,0,0,0,0]])

array([298.44607641])

## Checking for Adj - r2

In [30]:
def adj_r2(x,y,lr):
    r2 = lr.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adj_r2

adj_r2_lr = adj_r2(x_test,y_test,lr)

In [31]:
adj_r2_lr

0.7956259531901211

## Lasso Regularization

In [32]:
lassocv = LarsCV(cv=10,max_iter=10000,normalize=True)
lassocv.fit(x_train,y_train)

LarsCV(cv=10, max_iter=10000)

In [33]:
lassocv.alpha_

0.00012045211114874764

In [34]:
lasso = Lasso(alpha=lassocv.alpha_)
lasso.fit(x_train,y_train)

Lasso(alpha=0.00012045211114874764)

In [35]:
lasso.score(x_test,y_test)

0.7965765428272784

In [36]:
adj_r2_lasso = adj_r2(x_test,y_test,lasso)

In [37]:
adj_r2_lasso

0.7956565372420752

## Ridge regularization

In [38]:
ridgecv = RidgeCV(cv=5,normalize=True)
ridgecv.fit(x_train,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=5, normalize=True)

In [39]:
ridgecv.alpha_

0.1

In [40]:
ridge_lr = Ridge(alpha=ridgecv.alpha_)
ridge_lr.fit(x_train,y_train)

Ridge(alpha=0.1)

In [41]:
ridge_lr.score(x_test,y_test)

0.7965458837134907

In [42]:
adj_r2_ridge = adj_r2(x_test,y_test,ridge_lr)

In [43]:
adj_r2_ridge

0.7956257394689787

## Elastic net 

In [44]:
elastic = ElasticNetCV(alphas=None,cv=10)
elastic.fit(x_train,y_train)

ElasticNetCV(cv=10)

In [45]:
elastic.alpha_

0.011924026731250002

In [46]:
elastic.l1_ratio_

0.5

In [47]:
elastic_lr = ElasticNet(alpha=elastic.alpha_,l1_ratio=elastic.l1_ratio_)
elastic_lr.fit(x_train,y_train)

ElasticNet(alpha=0.011924026731250002)

In [48]:
elastic_lr.score(x_test,y_test)

0.7940771148863446

In [49]:
adj_r2_elastic = adj_r2(x_test,y_test,elastic_lr)

In [50]:
adj_r2_elastic

0.7931458053556798

In [51]:
score_df = {"LR" : adj_r2_lr,"Lasso" : adj_r2_lasso,"Ridge" : adj_r2_ridge,"Elastic" :adj_r2_elastic}
score_df

{'LR': 0.7956259531901211,
 'Lasso': 0.7956565372420752,
 'Ridge': 0.7956257394689787,
 'Elastic': 0.7931458053556798}

In [52]:
pickle.dump(lasso,open('Ai4i_lasso_final.pickle','wb'))

In [53]:
pickle.dump(scaler,open('scaler_std.pickle','wb'))

NameError: name 'scaler' is not defined

In [54]:
df.head(100)

,AirtemperatureK,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,TWF,HDF,PWF,OSF,RNF
0,298.1,308.6,1551,42.8,0,0,0,0,0,0
1,298.2,308.7,1408,46.3,3,0,0,0,0,0
2,298.1,308.5,1498,49.4,5,0,0,0,0,0
3,298.2,308.6,1433,39.5,7,0,0,0,0,0
4,298.2,308.7,1408,40.0,9,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
95,299.0,309.0,1351,52.2,44,0,0,0,0,0
96,299.0,309.0,1575,35.3,47,0,0,0,0,0
97,298.9,308.9,1750,29.9,50,0,0,0,0,0
98,298.9,308.8,1529,32.7,53,0,0,0,0,0


In [55]:
df.loc[df['AirtemperatureK'] > 310.2]

,AirtemperatureK,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,TWF,HDF,PWF,OSF,RNF


In [56]:
m = pickle.load(open("Ai4i_lasso_final.pickle",'rb'))

In [62]:
m.predict([[500.7,9500,40.2,30,0,0,1,1,0]])

array([523.90382113])